In [1]:
import pandas as pd
import numpy as np
from functools import reduce

In [2]:
data = pd.read_csv('data/diabetes.csv')

In [3]:
OUTCOME = 'Outcome'

target = data[OUTCOME]
data_without_target = data.loc[:, data.columns != OUTCOME]

In [4]:
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [5]:
data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [6]:
def avg_utmost_values(column):
    """
    Метод для определения полусуммы крайних наблюдений
    
    :param column: Series
    return float
    """
    return (column.min() + column.max()) / 2

In [7]:
def avg_absolute_deviations(column):
    """
    Метод для определения среднего модуля отклонений
    
    :param column: Series instance
    return float
    """
    median = column.median()
    return np.sum(np.absolute(column.apply(lambda value: value - median) / column.size))

In [8]:
def scope(column):
    """
    Метод для определения размаха
    
    :param column: Series instance
    return float
    """
    return column.max() - column.min()

In [9]:
def normilize(data):
    """
    Метод для нормализации и центрирования данных
    
    :param data: DataFrame instance
    return DataFrame instance
    """
    def encode_by_column(column):
        mean, std = column.mean(), column.std()
        return column.apply(lambda x: (x - mean) / std)
    
    return data.apply(encode_by_column)

In [10]:
def encode_on_hypercube(data):
    """
    Метод для кодирования на гиперкуб
    
    :param data: DataFrame instance
    return DataFrame instance
    """
    def encode_by_column(column):
        minimum, maximum = column.min(), column.max()
        return column.apply(lambda x: 2 * (x - minimum) / (maximum - minimum) - 1)
    
    return data.apply(encode_by_column)

In [11]:
def encode_on_hyperball(data):
    """
    Метод для кодирования на гипершар
    
    :param data: DataFrame instance
    return DataFrame instance
    """
    def encode_by_column(column):
        minimum, maximum = column.min(), column.max()
        return column.apply(lambda x: (x - minimum) / (maximum - minimum))
    
    return data.apply(encode_by_column)

In [12]:
def recurrent_relationship_of_mean(data):
    """
    Метод для рекуррентного соотношения среднего значения
    
    :param data: DataFrame instance
    return Series instance
    """
    rows = data.iterrows()
    _, mean = next(rows)
    number = 1
    
    for _, row in rows:
        number += 1
        mean += (row - mean) / number
    
    return mean

In [13]:
def recurrent_relationship_of_median(data):
    """
    Метод для рекуррентного соотношения медианы
    
    :param data: DataFrame instance
    return Series instance
    """
    rows = data.iterrows()
    _, median = next(rows)
    number = 1
    
    for _, row in rows:
        number += 1
        median += np.sign(row - median) / number
    
    return median

In [14]:
def print_info_about_column(label, column):
    """
    Метод для отображение базовой информации о конкретном параметре
    
    :param label: str, name of the column
    :param column: Series instance
    """
    print(f'Первичный анализ параметра {label}:')
    print(f'\tМат. ожидание: {column.mean()}')
    print(f'\tМедиана: {column.median()}')
    print(f'\tПолусумма крайних наблюдений: {avg_utmost_values(column)}')
    print(f'\tСреднеквадратическое отклонение: {column.std()}')
    print(f'\tСредний модуль отклонений: {avg_absolute_deviations(column)}')
    print(f'\tРазмах: {scope(column)}')
    print(f'\tДисперсия: {column.var()}')
    
    print(f'\tМинимальное значение: {column.min()}')
    print(f'\tМаксимальное значение: {column.max()}')

In [15]:
def print_info_about_data(data):
    """
    Метод для отображения базовой информации по каждой колонке датасета
    
    :param data: DataFrame instance
    """
    for label, column in data.iteritems():
        print_info_about_column(label, column)
        print('^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')
        
    print('Нормализированный и отцентрированный датасет:')
    print(normilize(data))

    print('------------------------------------')
    print('Закодированный на гипершар датасет:')
    print(encode_on_hyperball(data))

    print('------------------------------------')
    print('Закодированный на гиперкуб датасет:')
    print(encode_on_hypercube(data))
    
    print('------------------------------------')
    print('Рекуррентное соотношение медианы:')
    print(recurrent_relationship_of_median(data))
    
    print('------------------------------------')
    print('Рекуррентное соотношение среднего значения:')
    print(recurrent_relationship_of_mean(data))

    print('####################################')

In [16]:
# Отображение информации по всему датасету
print(f'Count of rows is {data_without_target.shape[0]}')
print_info_about_data(data_without_target)

Count of rows is 768
Первичный анализ параметра Pregnancies:
	Мат. ожидание: 3.8450520833333335
	Медиана: 3.0
	Полусумма крайних наблюдений: 8.5
	Среднеквадратическое отклонение: 3.3695780626988623
	Средний модуль отклонений: 2.68359375
	Размах: 17
	Дисперсия: 11.35405632062142
	Минимальное значение: 0
	Максимальное значение: 17
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Первичный анализ параметра Glucose:
	Мат. ожидание: 120.89453125
	Медиана: 117.0
	Полусумма крайних наблюдений: 99.5
	Среднеквадратическое отклонение: 31.97261819513622
	Средний модуль отклонений: 24.985677083333336
	Размах: 199
	Дисперсия: 1022.2483142519557
	Минимальное значение: 0
	Максимальное значение: 199
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Первичный анализ параметра BloodPressure:
	Мат. ожидание: 69.10546875
	Медиана: 72.0
	Полусумма крайних наблюдений: 61.0
	Среднеквадратическое отклонение: 19.355807170644777
	Средний модуль отклонений: 12.438802083333334
	Размах: 122
	Дисперсия: 374.6472712271838
	Минимальное значе

In [17]:
# Отображение информации по датасету, включающему информацию только о людях с подтвержденным диабетом
outcome_true_dataset = data[data[OUTCOME] == 1].loc[:, data.columns != OUTCOME]
print(f'Count of rows is {outcome_true_dataset.shape[0]}')
print_info_about_data(outcome_true_dataset)

Count of rows is 268
Первичный анализ параметра Pregnancies:
	Мат. ожидание: 4.865671641791045
	Медиана: 4.0
	Полусумма крайних наблюдений: 8.5
	Среднеквадратическое отклонение: 3.741239044041554
	Средний модуль отклонений: 3.1343283582089554
	Размах: 17
	Дисперсия: 13.996869584660963
	Минимальное значение: 0
	Максимальное значение: 17
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Первичный анализ параметра Glucose:
	Мат. ожидание: 141.25746268656715
	Медиана: 140.0
	Полусумма крайних наблюдений: 99.5
	Среднеквадратическое отклонение: 31.939622058007195
	Средний модуль отклонений: 25.936567164179102
	Размах: 199
	Дисперсия: 1020.1394572083399
	Минимальное значение: 0
	Максимальное значение: 199
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Первичный анализ параметра BloodPressure:
	Мат. ожидание: 70.82462686567165
	Медиана: 74.0
	Полусумма крайних наблюдений: 57.0
	Среднеквадратическое отклонение: 21.49181165060413
	Средний модуль отклонений: 13.279850746268657
	Размах: 114
	Дисперсия: 461.8979680250434

In [18]:
# Отображение информации по датасету, включающему информацию только о людях с неподтвержденным диабетом
outcome_false_dataset = data[data[OUTCOME] == 0].loc[:, data.columns != OUTCOME]
print(f'Count of rows is {outcome_false_dataset.shape[0]}')
print_info_about_data(outcome_false_dataset)

Count of rows is 500
Первичный анализ параметра Pregnancies:
	Мат. ожидание: 3.298
	Медиана: 2.0
	Полусумма крайних наблюдений: 6.5
	Среднеквадратическое отклонение: 3.01718458262189
	Средний модуль отклонений: 2.306
	Размах: 13
	Дисперсия: 9.103402805611228
	Минимальное значение: 0
	Максимальное значение: 13
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Первичный анализ параметра Glucose:
	Мат. ожидание: 109.98
	Медиана: 107.0
	Полусумма крайних наблюдений: 98.5
	Среднеквадратическое отклонение: 26.14119975535359
	Средний модуль отклонений: 19.772000000000002
	Размах: 197
	Дисперсия: 683.3623246492986
	Минимальное значение: 0
	Максимальное значение: 197
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Первичный анализ параметра BloodPressure:
	Мат. ожидание: 68.184
	Медиана: 70.0
	Полусумма крайних наблюдений: 61.0
	Среднеквадратическое отклонение: 18.063075413305828
	Средний модуль отклонений: 11.788000000000002
	Размах: 122
	Дисперсия: 326.2746933867735
	Минимальное значение: 0
	Максимальное значение: 1